In [103]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:85% !important; }</style>"))

In [104]:
import os
import time
import numpy as np
import pandas as pd

from os import listdir
from io import BytesIO
import requests

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers,models,utils
from tensorflow.keras.layers import Dense,Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from scipy import stats
from sklearn import preprocessing

from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, auc

import PIL
from PIL import Image

import seaborn as sns
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt

In [105]:
if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

Please install GPU version of TF


In [106]:
DATA_DIR = 'data/caps and shoes/'
GEN_DATA_DIR = 'data/caps and shoes generated/'

In [107]:
def convert_img_to_data(image):
    data = np.asarray(image)
    gs_image = image.convert(mode='L')
    gs_data = np.asarray(gs_image)
    gs_resized = gs_image.resize((112,112))
    gs_resized_data = np.asarray(gs_resized)
    reshaped_gs_data = gs_resized_data.reshape(112*112)
    return reshaped_gs_data

def convert_images_from_dir(dir_path):
    image_data = []
    
    for filename in listdir(dir_path):
        image = Image.open(dir_path +os.sep + filename)
        reshaped_gs_data = convert_img_to_data(image)
        image_data.append(reshaped_gs_data)
    
    return image_data

def load_from_dir(dir_path, labels):
    label_data = []
    image_data = []
    for label in labels:
        data_from_dir = convert_images_from_dir(dir_path + label)
        labels_for_data = [label for i in range(len(data_from_dir))]
        image_data += data_from_dir
        label_data += labels_for_data
    
    print('Found %d images belonging to %d classes' % (len(image_data),  len(labels)))
    return (np.array(image_data),np.array(label_data))

def load_img_data(data_dir):
    train_dir = data_dir + 'train/'
    validation_dir = data_dir + 'val/'
    test_dir = data_dir + 'test/'
    
    if (os.path.isdir(train_dir) and os.path.isdir(validation_dir) and os.path.isdir(test_dir)) :
        labels = [subdirname.name for subdirname in os.scandir(train_dir) if subdirname.is_dir()] 
        
        train_data = load_from_dir(train_dir,labels)
        validation_data = load_from_dir(validation_dir,labels)
        test_data = load_from_dir(test_dir,labels)
        
    return train_data, validation_data, test_data     

def url_to_image(img_url):
    response = requests.get(img_url)
    img = Image.open(BytesIO(response.content))
    return img

def plot_images(ims, figsize=(24,12), rows=1, interp=False, titles=None):
    f = plt.figure(figsize=figsize)
    cols = len(ims)//rows if len(ims) % 2 == 0 else len(ims)//rows + 1
    for i in range(len(ims)):
        sp = f.add_subplot(rows, cols, i+1)
        sp.axis('Off')
        if titles is not None:
            sp.set_title(titles[i], fontsize=18)
        
        plt.imshow(ims[i].reshape(112,112), cmap="gray")

In [108]:
train_data, validation_data, test_data = load_img_data(DATA_DIR)
X_train, y_train = train_data
X_val, y_val = validation_data
X_test, y_test = test_data

Found 200 images belonging to 2 classes
Found 100 images belonging to 2 classes
Found 50 images belonging to 2 classes


In [109]:
X_train = X_train.astype('float32') / 255
X_val = X_val.astype('float32') / 255
X_test = X_test.astype('float32') / 255

In [110]:
le = preprocessing.LabelEncoder()
le.fit(y_train)
y_train = le.transform(y_train)
y_val = le.transform(y_val)
y_test = le.transform(y_test)
y_train = utils.to_categorical(y_train)
y_val = utils.to_categorical(y_val)
y_test = utils.to_categorical(y_test)

In [111]:
def define_multilayer_model_architecture():
    model = models.Sequential()
    model.add(Dense(32, activation='relu', input_shape=(12544,)))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(2, activation='softmax'))
    
    model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [112]:
ITER = 5
training_time_list = []
test_accuracy_list = []
for iter_count in range(ITER):
    model = define_multilayer_model_architecture()
    start_time = time.time()
    model.fit(X_train, y_train, validation_data = (X_val,y_val), epochs=125, batch_size=200, verbose=0, shuffle=True)
    training_time = time.time() - start_time
    training_time_list.append(training_time)
    test_loss, test_accuracy = model.evaluate(X_test, y_test, batch_size=50)
    test_accuracy_list.append(test_accuracy)

print('Avg training time : %.3f s' % np.mean(training_time_list))
print('Avg test accuracy : %.4f +- %.2f' % (np.mean(test_accuracy_list), np.std(test_accuracy_list)))
print('Total parameters : %d' % model.count_params())

50/50 [==============================] - 0s 359us/sample - loss: 0.5136 - accuracy: 0.7000
Avg training time : 1.323 s
Avg test accuracy : 0.7120 +- 0.04
Total parameters : 402002


In [113]:
def define_singlelayer_model_architecture(no_of_neurons=32):
    model = models.Sequential()
    model.add(Dense(no_of_neurons, activation='relu', input_shape=(12544,)))
    model.add(Dense(2, activation='softmax'))
    
    model.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [114]:
ITER = 5
training_time_list = []
test_accuracy_list = []
for iter_count in range(ITER):
    model = define_singlelayer_model_architecture(32)
    start_time = time.time()
    model.fit(X_train, y_train, validation_data = (X_val,y_val), epochs=125, batch_size=200, verbose=0, shuffle=True)
    training_time = time.time() - start_time
    training_time_list.append(training_time)
    test_loss, test_accuracy = model.evaluate(X_test, y_test, batch_size=50)
    test_accuracy_list.append(test_accuracy)

print('Avg training time : %.3f s' % np.mean(training_time_list))
print('Avg test accuracy : %.4f +- %.2f' % (np.mean(test_accuracy_list), np.std(test_accuracy_list)))
print('Total parameters : %d' % model.count_params())

50/50 [==============================] - 0s 283us/sample - loss: 0.5386 - accuracy: 0.7000
Avg training time : 1.246 s
Avg test accuracy : 0.7400 +- 0.02
Total parameters : 401506


### Learn from generated Images

In [115]:
train_data, validation_data, test_data = load_img_data(GEN_DATA_DIR)
X_train, y_train = train_data
X_val, y_val = validation_data
X_test, y_test = test_data

Found 1000 images belonging to 2 classes
Found 500 images belonging to 2 classes
Found 50 images belonging to 2 classes


In [116]:
X_train = X_train.astype('float32') / 255
X_val = X_val.astype('float32') / 255
X_test = X_test.astype('float32') / 255

In [117]:
le = preprocessing.LabelEncoder()
le.fit(y_train)
y_train = le.transform(y_train)
y_val = le.transform(y_val)
y_test = le.transform(y_test)
y_train = utils.to_categorical(y_train)
y_val = utils.to_categorical(y_val)
y_test = utils.to_categorical(y_test)

In [118]:
ITER = 5
training_time_list = []
test_accuracy_list = []
for iter_count in range(ITER):
    model = define_multilayer_model_architecture()
    start_time = time.time()
    model.fit(X_train, y_train, validation_data = (X_val,y_val), epochs=125, batch_size=500, verbose=0, shuffle=True)
    training_time = time.time() - start_time
    training_time_list.append(training_time)
    test_loss, test_accuracy = model.evaluate(X_test, y_test, batch_size=50)
    test_accuracy_list.append(test_accuracy)

print('Avg training time : %.3f s' % np.mean(training_time_list))
print('Avg test accuracy : %.4f +- %.2f' % (np.mean(test_accuracy_list), np.std(test_accuracy_list)))
print('Total parameters : %d' % model.count_params())

50/50 [==============================] - 0s 379us/sample - loss: 0.5483 - accuracy: 0.6600
Avg training time : 3.994 s
Avg test accuracy : 0.6560 +- 0.08
Total parameters : 402002


In [119]:
ITER = 5
training_time_list = []
test_accuracy_list = []
for iter_count in range(ITER):
    model = define_singlelayer_model_architecture(32)
    start_time = time.time()
    model.fit(X_train, y_train, validation_data = (X_val,y_val), epochs=125, batch_size=500, verbose=0, shuffle=True)
    training_time = time.time() - start_time
    training_time_list.append(training_time)
    test_loss, test_accuracy = model.evaluate(X_test, y_test, batch_size=50)
    test_accuracy_list.append(test_accuracy)

print('Avg training time : %.3f s' % np.mean(training_time_list))
print('Avg test accuracy : %.4f +- %.2f' % (np.mean(test_accuracy_list), np.std(test_accuracy_list)))
print('Total parameters : %d' % model.count_params())

50/50 [==============================] - 0s 400us/sample - loss: 0.5523 - accuracy: 0.7600
Avg training time : 4.477 s
Avg test accuracy : 0.7440 +- 0.06
Total parameters : 401506
